# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

## Import Libraries

In [24]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [2]:
#!apt-get update && apt-get install -y python3-opencv

In [3]:
#!pip install sklearn

In [98]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

#MBConvBlock
import math
from functools import partial
from torch.nn import functional as F

#ScaledDotScaling
from torch.nn import init

In [99]:
import wandb

In [100]:
wandb.init(project="my-test-project", entity="jch")

In [101]:
wandb.config = {
  "learning_rate": 0.05,
  "epochs": 50,
  "batch_size": 32
}

## Set Arguments & hyperparameters

In [102]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [103]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= 'data'
NUM_CLS = 2

EPOCHS = 50
BATCH_SIZE = 32 #32
LEARNING_RATE = 0.05 #0.0005
EARLY_STOPPING_PATIENCE = 10
INPUT_SHAPE = 128 # 128

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# DEVICE=torch.device('cpu')

## Dataloader

#### Train & Validation Set loader

In [104]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.9)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.9):]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))
        return trans_image, data['COVID']


In [110]:
# import torch.nn.functional as F

# class custom_CNN(nn.Module):
#     def __init__(self, num_classes):
#         super(custom_CNN, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
#         self.pool = nn.MaxPool2d(kernel_size=2)
#         self.conv2 = nn.Conv2d(in_channels=8, out_channels=25, kernel_size=5)
#         self.conv3 = nn.Conv2d(in_channels=25, out_channels=40, kernel_size=5)
        
#         self.fc1 = nn.Linear(in_features=40*12*12, out_features=128)
#         self.fc2 = nn.Linear(in_features=128, out_features=num_classes)
#         self.softmax = nn.Softmax(dim=1)
        
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x))) # (32, 3, 128, 128) -> (32, 8, 62, 62)
#         x = self.pool(F.relu(self.conv2(x))) # (32, 8, 62, 62) -> (32, 25, 29, 29)
#         x = self.pool(F.relu(self.conv3(x))) # (32, 25, 29, 29) -> (32, 40, 12, 12)
        
#         # print(x.shape)
#         x = torch.flatten(x,1)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
        
#         output = self.softmax(x)
#         # print(output)
#         return output

## Utils
### EarlyStopper

In [111]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        # print(msg)

### Trainer

In [112]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
            wandb.log({"loss": loss})
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

### Metrics

In [113]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [114]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))
# print(train_dataset[1][0].shape)

Loading train dataset..
Loading val dataset..
Train set samples: 581 Val set samples: 65


#### Load model and other utils

In [115]:
# Load Model
# model = custom_CNN(NUM_CLS).to(DEVICE)
model = CoAtNet(3, 128).to(DEVICE)
# model = COVID19Model(pretrained=False)

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

In [116]:
model

CoAtNet(
  (maxpool2d): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (maxpool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (s0): Sequential(
    (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (mlp0): Sequential(
    (0): Conv2d(3, 4, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(4, 4, kernel_size=(1, 1), stride=(1, 1))
  )
  (s1): MBConvBlock(
    (_depthwise_conv): Conv2dStaticSamePadding(
      4, 4, kernel_size=(3, 3), stride=(1, 1), groups=4, bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn1): BatchNorm2d(4, eps=0.01, momentum=0.1, affine=True, track_running_stats=True)
    (_se_reduce): Conv2dStaticSamePadding(
      4, 1, kernel_size=(1, 1), stride=(1, 1)
      (static_padding): Identity()
    )
    (_se_expand): Conv2dStaticSamePaddi

### epoch 단위 학습 진행

In [117]:
for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)

    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'best.pt')

  0% 0/50 [00:00<?, ?it/s]/tmp/ipykernel_62782/1260797069.py:17: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  i = ctx.saved_variables[0]


Epoch 0, Train loss: 0.7292079958650801, Acc: 0.5335628227194492, F1-Macro: 0.34792368125701456


  2% 1/50 [00:41<34:12, 41.89s/it]

Epoch 0, Val loss: 1.01891428232193, Acc: 0.49230769230769234, F1-Macro: 0.32989690721649484


/tmp/ipykernel_62782/1260797069.py:17: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  i = ctx.saved_variables[0]


...

Epoch 19, Train loss: 0.7302143971125284, Acc: 0.5335628227194492, F1-Macro: 0.34792368125701456
Epoch 19, Val loss: 1.0109850764274597, Acc: 0.49230769230769234, F1-Macro: 0.32989690721649484


 40% 20/50 [13:50<20:41, 41.37s/it]/tmp/ipykernel_62782/1260797069.py:17: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  i = ctx.saved_variables[0]


Epoch 20, Train loss: 0.7276526126596663, Acc: 0.5335628227194492, F1-Macro: 0.34792368125701456


 40% 20/50 [14:31<21:46, 43.57s/it]

Epoch 20, Val loss: 1.0761368572711945, Acc: 0.49230769230769234, F1-Macro: 0.32989690721649484
Early stopped


## Inference
### 모델 로드

In [118]:
TRAINED_MODEL_PATH = 'best.pt'

### Load dataset

In [119]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [120]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


### 추론 진행

In [121]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

1it [00:01,  1.99s/it]

tensor([[0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656]], device='cuda:0')


2it [00:03,  1.99s/it]

tensor([[0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656]], device='cuda:0')


3it [00:06,  2.02s/it]

tensor([[0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656]], device='cuda:0')


4it [00:06,  1.59s/it]

tensor([[0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656],
        [0.5344, 0.4656]], device='cuda:0')


### 결과 저장

In [122]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('prediction.csv', index=False)